# Scrapping from website

The main task is to write a script to get a nice CSV file of natural gas prices.
We includ both script and resulting data with CSV file.

More detail:

    1. Prices should be Henry Hub gas prices from EIA data here: http://www.eia.gov/dnav/ng/hist/rngwhhdm.htm

    2. Main data wanted is daily prices.
        Bonus points for doing other granularities (e.g. month) - do them in separate CSV files with sensible naming
        
    3. Resulting CSV should have two columns: Date and Price. You may need to normalize the data to get this and/or work out dates. For months the Date should be the first date of the month.
    
    4. We want a script for this and we want this script to be in python (we'd allow node or bash or go script at a push but prefer python)
        Why a script? Ans: We'll want to run this again and again as they release new data. You could copy and paste data into Excel/Google Docs by hand, and then export the CSV. But that would be tedious, time consuming and error prone to do month after month. We use simple python libraries wherever possible: `requests`, `csv`, `xlrd`, and tools as csvkit and messytables.

For installing some of these tools one can simply use 


    pip install requests
    pip install pandas



Data source, we are working with: https://www.eia.gov/dnav/ng/hist/rngwhhdm.htm

# Monthly price 
First we get monthly price evolution data from the data source on the page (we use Hint that we can directly copy the link of the file from the page). The content file will be updated with time, but the link we keep.

Algorithm:
1. first get xls file 
2. get csv file in requested form

In [2]:
import requests
dls = "https://www.eia.gov/dnav/ng/hist_xls/RNGWHHDm.xls"
resp = requests.get(dls)

output = open('monthlyprice.xls', 'wb')
output.write(resp.content)
output.close()

In [10]:
# first way to convert xls to csv
import pandas as pd

data = pd.read_excel (r'monthlyprice.xls',sheet_name='Data 1')
data.head()


,Back to Contents,Data 1: Henry Hub Natural Gas Spot Price (Dollars per Million Btu)
0,Sourcekey,RNGWHHD
1,Date,Henry Hub Natural Gas Spot Price (Dollars per ...
2,1997-01-15 00:00:00,3.45
3,1997-02-15 00:00:00,2.15
4,1997-03-15 00:00:00,1.89


Important comment before we save csv comma separated file format:
    1. we see that first column is in date format (each month), 
    2. we see that first row is not neccesary so we need to change it for the late document we save to csv from pandas

# Data cleaning 
We want all data csv files to be in format of 
            first column (time), second column (price) starting from the first row (column names) and then data.

In [19]:
from datetime import datetime


# delet first rows of data 
data = data.iloc[2:]

print(data.columns)
#rename columns 
data.columns = ['Date', 'Price']

# make column Date in datetime format
data['Date'] = pd.to_datetime(data['Date'])

data.head()

Index(['Date', 'Price'], dtype='object')


,Date,Price
15,1998-02-15,2.23
16,1998-03-15,2.24
17,1998-04-15,2.43
18,1998-05-15,2.14
19,1998-06-15,2.17


In [20]:
data.to_csv(r'monthlyprice_1.csv')#, index = None, header=True)


In [9]:
# second way to transform XLS to CSV 


import xlrd
import csv

def csv_from_xsl(wb,sh):
    ''' given names of xls file and sheet name sh produce csv files'''

    your_csv_file = open('monthlyprice.csv', 'w', encoding='utf8')
    wr = csv.writer(your_csv_file, quoting=csv.QUOTE_ALL)

    for rownum in range(sh.nrows):
        wr.writerow(sh.row_values(rownum))

    your_csv_file.close()
    
    


wb = xlrd.open_workbook('monthlyprice.xls')
sh = wb.sheet_by_name('Data 1')

csv_from_xsl(wb,sh)   
    

<_io.TextIOWrapper name='monthlypricetest.csv' mode='w' encoding='utf8'>

# Data for daily price evolution 
We want to have other granularities (e.g. month) - do them in separate CSV files with corresponding naming `dailyprice.csv`.

In [22]:
# function to get daily price 





def dailyprice():
    
    # monthly price
    dls = "https://www.eia.gov/dnav/ng/hist_xls/RNGWHHDd.xls"
    resp = requests.get(dls)
    
    output = open('dailyprice.xls', 'wb')
    output.write(resp.content)
    output.close()
    
    
    
    data = pd.read_excel (r'dailyprice.xls',sheet_name='Data 1')
    data.head()
    
    
    # delet first rows of data 
    data = data.iloc[2:]
    
    print(data.columns)
    #rename columns 
    data.columns = ['Date', 'Price']
    
    # make column Date in datetime format
    data['Date'] = pd.to_datetime(data['Date'])
    data.head()
    
    data.to_csv(r'dailyprice.csv')#, index = None, header=True)    
    
    return data


#execute and get dailyprice.csv
dailyprice()




Index(['Back to Contents', 'Data 1: Henry Hub Natural Gas Spot Price (Dollars per Million Btu)'], dtype='object')


,Date,Price
2,1997-01-07,3.82
3,1997-01-08,3.8
4,1997-01-09,3.61
5,1997-01-10,3.92
6,1997-01-13,4
...,...,...
5829,2020-03-09,1.74
5830,2020-03-10,1.91
5831,2020-03-11,1.96
5832,2020-03-12,1.82


# Using beautifulsoup

There are lots of ways to do the same procedure, one is to use beautifulsoup tool, which works well for many known websites, e.g. for wikipedia, twitter and others.

In [4]:
import os
from bs4 import BeautifulSoup
# Python 3.x
from urllib.request import urlopen, urlretrieve

URL = 'https://www.eia.gov/dnav/ng/hist/rngwhhdm.htm'
OUTPUT_DIR = ''  # path to output folder, '.' or '' uses current folder

u = urlopen(URL)
try:
    html = u.read().decode('utf-8')
finally:
    u.close()


soup = BeautifulSoup(html, "html.parser")
for link in soup.select('a[href^="http://"]'):
    href = link.get('href')
    if not any(href.endswith(x) for x in ['.csv','.xls','.xlsx']):
        continue

    filename = os.path.join(OUTPUT_DIR, href.rsplit('/', 1)[-1])

    # We need a https:// URL for this site
    href = href.replace('http://','https://')

    print("Downloading %s to %s..." % (href, filename) )
    urlretrieve(href, filename)
    print("Done.")

# Datapackage 

Below we also create a datapackage.json file.

The datapackage.json is a small file in JSON format that gives a bit of information about your dataset. You’ll need to create this file and then place it in the directory you created.
Don't worry if you don't know what JSON is - we provide some tools that can automatically create your this file for you.

There are three options for creating the datapackage.json:

Option 1: Use the online datapackage.json creator tool - just answer a few questions and give it your data files and it will spit out a datapackage.json for you to include in your project

Option 2: Do it yourself - if you’re familiar with JSON you can just create this yourself. Take a look at the Data Package and Simple Data Format specs https://datahub.io/docs/data-packages 